# Задание по продуктовой аналитике

In [49]:
import pandas as pd

pd.set_option('display.max_columns', None)
deals_df = pd.read_pickle("deals_cleaned.pkl")
deals_df

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range
0,5805028000056864695,Ben Hall,NaT,NaN,New Lead,NaN,03.07.23women,NaT,facebook ads,NaN,NaN,NaN,2024-06-21 15:30:00,<NA>,<NA>,NaN,NaN,5805028000056849495,NaN,NaN,NaN,NaN,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaT,NaN,New Lead,NaN,NaN,NaT,organic,NaN,Web Developer,Morning,2024-06-21 15:23:00,6,<NA>,NaN,2000.0,5805028000056834471,NaN,NaN,NaN,NaN,NaN,NaN
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,engwien_at,0 days 00:26:43,telegram posts,NaN,NaN,NaN,2024-06-21 14:45:00,<NA>,<NA>,NaN,NaN,5805028000056854421,NaN,NaN,0.0,NaN,NaN,Medium (15-30 min)
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,04.07.23recentlymoved_de,0 days 01:00:04,facebook ads,NaN,NaN,NaN,2024-06-21 13:32:00,<NA>,<NA>,NaN,NaN,5805028000056889351,NaN,NaN,0.0,NaN,NaN,Very slow (>60 min)
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,discovery_de,0 days 00:53:12,google ads,NaN,NaN,NaN,2024-06-21 13:21:00,<NA>,<NA>,NaN,NaN,5805028000056876176,NaN,NaN,0.0,NaN,NaN,Slow (30-60 min)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21406,5805028000050001603,Quincy Vincent,2024-05-15,E - Non Qualified,Lost,Duplicate,NaN,0 days 05:35:41,organic,NaN,NaN,NaN,2024-05-15 08:50:00,<NA>,<NA>,NaN,NaN,5805028000049855125,NaN,NaN,0.0,NaN,NaN,Very slow (>60 min)
21407,5805028000053787195,Paula Underwood,2024-06-06,E - Non Qualified,Lost,Duplicate,NaN,NaT,organic,NaN,NaN,NaN,2024-06-06 18:45:00,<NA>,<NA>,NaN,NaN,5805028000053705642,NaN,NaN,0.0,NaN,NaN,NaN
21408,5805028000054823001,Quincy Vincent,2024-06-12,E - Non Qualified,Lost,Duplicate,brand_search_eng_de,0 days 04:30:17,google ads,NaN,NaN,NaN,2024-06-12 05:46:00,<NA>,<NA>,NaN,NaN,5805028000054480760,NaN,NaN,0.0,NaN,NaN,Very slow (>60 min)
21409,5805028000056685164,Charlie Davis,2024-06-20,E - Non Qualified,Lost,Duplicate,NaN,0 days 00:01:46,organic,NaN,NaN,NaN,2024-06-20 13:49:00,<NA>,<NA>,NaN,NaN,5805028000056392363,NaN,NaN,0.0,NaN,NaN,Very fast (≤5 min)


In [50]:
deals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Id                    21411 non-null  object         
 1   Deal Owner Name       21382 non-null  object         
 2   Closing Date          16685 non-null  datetime64[ns] 
 3   Quality               19158 non-null  category       
 4   Stage                 21411 non-null  category       
 5   Lost Reason           15942 non-null  category       
 6   Campaign              16006 non-null  object         
 7   SLA                   15504 non-null  timedelta64[ns]
 8   Source                21411 non-null  object         
 9   Payment Type          496 non-null    category       
 10  Product               3592 non-null   category       
 11  Education Type        3299 non-null   category       
 12  Created Time          21411 non-null  datetime64[ns] 
 13  C

In [51]:
deals_df['Course duration'].nunique()

2

$$
AOV_i = \frac{\text{Initial Amount Paid} + \left(\text{Offer Total Amount} - \text{Initial Amount Paid}\right)}{\text{Course duration} - 1} \cdot (\text{Months of study} - 1) \div \text{Months of study}
$$

In [52]:
import numpy as np

# Условия для валидных строк
valid = (
    deals_df['Initial Amount Paid'].notna() &
    deals_df['Offer Total Amount'].notna() &
    deals_df['Course duration'].notna() &
    deals_df['Months of study'].notna() &
    (deals_df['Course duration'] != 1) &
    (deals_df['Months of study'] != 0)
)

# Расчёт AOV_i с приведением к float64
deals_df['AOV_i'] = np.where(
    valid,
    (
        deals_df['Initial Amount Paid'] +
        ((deals_df['Offer Total Amount'] - deals_df['Initial Amount Paid']) /
         (deals_df['Course duration'] - 1)) *
        (deals_df['Months of study'] - 1)
    ) / deals_df['Months of study'],
    np.nan
).astype('float64') #Т.к. тип данных получается Float64 (с большой буквы), то это может испортить данные (возвращает pd.NA вместо pd.nan и ломает вычисления даже при отсутствии строк с NaN), поэтому меняем тип на float64:

In [53]:
deals_df[deals_df['AOV_i'].notna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i
463,5805028000055502703,Eva Kent,2024-06-18,B - Medium,Payment Done,NaN,performancemax_wide_at,0 days 14:35:20,google ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 20:01:00,11,1,500.0,4500.0,5805028000055471387,Podskalie,NaN,3.0,48.1667,18.3167,Very slow (>60 min),500.000000
473,5805028000055524315,Ben Hall,2024-06-19,A - High,Payment Done,NaN,12.07.2023wide_de,0 days 00:17:25,tiktok ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 17:02:00,11,1,500.0,4500.0,5805028000055503302,Zinnowitz,NaN,4.0,54.0777,13.9104,Medium (15-30 min),500.000000
491,5805028000055524040,Ben Hall,2024-06-23,B - Medium,Payment Done,NaN,live_de,0 days 00:23:42,telegram posts,NaN,Web Developer,Morning,2024-06-15 12:20:00,6,1,1000.0,9000.0,5805028000055464156,Wolfsburg,B1,8.0,52.4226,10.7865,Medium (15-30 min),1000.000000
824,5805028000054664018,Ben Hall,2024-06-19,D - Non Target,Payment Done,NaN,NaN,0 days 16:31:52,organic,NaN,Web Developer,Morning,2024-06-11 19:39:00,6,1,1000.0,9000.0,5805028000054683001,Zwickau,A2,8.0,50.7194,12.4884,Very slow (>60 min),1000.000000
1029,5805028000054257317,Quincy Vincent,2024-06-17,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 00:45:55,facebook ads,NaN,Digital Marketing,Morning,2024-06-10 15:33:00,11,1,1000.0,11000.0,5805028000054284461,Leinfelden-Echterdingen,B1,7.0,48.6941,9.1479,Slow (30-60 min),1000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21034,5805028000002302112,Julia Nelson,2023-07-19,C - Low,Payment Done,Expensive,performancemax_digitalmarkt_ru_de,0 days 15:24:52,google ads,NaN,Web Developer,Morning,2023-07-18 21:11:00,6,2,1000.0,5000.0,5805028000002302109,Wolfsburg,B1,1.0,52.4226,10.7865,Very slow (>60 min),900.000000
21087,5805028000001987082,Julia Nelson,2023-07-17,C - Low,Payment Done,Conditions are not suitable,12.07.2023wide_de,0 days 00:49:16,tiktok ads,NaN,Digital Marketing,Morning,2023-07-17 18:02:00,11,11,1000.0,11000.0,5805028000001986077,NaN,B1,0.0,NaN,NaN,Slow (30-60 min),1000.000000
21130,5805028000001885076,Jane Smith,2023-08-31,A - High,Payment Done,NaN,04.07.23recentlymoved_de,0 days 00:13:31,facebook ads,One Payment,Digital Marketing,Morning,2023-07-15 13:27:00,11,11,450.0,4000.0,5805028000001880249,Ingolstadt,NaN,47.0,48.7651,11.4237,Fast (5-15 min),363.636364
21265,5805028000001401001,Oliver Taylor,2023-07-16,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 02:22:36,facebook ads,NaN,Digital Marketing,NaN,2023-07-08 08:56:00,11,8,1000.0,11500.0,5805028000001350049,NaN,NaN,8.0,NaN,NaN,Very slow (>60 min),1043.750000


$$
Revenue_i = AOV_i \times \text{Months of study}
$$

In [54]:
deals_df['Revenue_i'] = (deals_df['AOV_i'] * deals_df['Months of study']).astype('float64') #Т.к. тип данных получается Float64 (с большой буквы), то это может испортить данные (возвращает pd.NA вместо pd.nan и ломает вычисления даже при отсутствии строк с NaN), поэтому меняем тип на float64:

In [55]:
deals_df[deals_df['AOV_i'].notna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i,Revenue_i
463,5805028000055502703,Eva Kent,2024-06-18,B - Medium,Payment Done,NaN,performancemax_wide_at,0 days 14:35:20,google ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 20:01:00,11,1,500.0,4500.0,5805028000055471387,Podskalie,NaN,3.0,48.1667,18.3167,Very slow (>60 min),500.000000,500.0
473,5805028000055524315,Ben Hall,2024-06-19,A - High,Payment Done,NaN,12.07.2023wide_de,0 days 00:17:25,tiktok ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 17:02:00,11,1,500.0,4500.0,5805028000055503302,Zinnowitz,NaN,4.0,54.0777,13.9104,Medium (15-30 min),500.000000,500.0
491,5805028000055524040,Ben Hall,2024-06-23,B - Medium,Payment Done,NaN,live_de,0 days 00:23:42,telegram posts,NaN,Web Developer,Morning,2024-06-15 12:20:00,6,1,1000.0,9000.0,5805028000055464156,Wolfsburg,B1,8.0,52.4226,10.7865,Medium (15-30 min),1000.000000,1000.0
824,5805028000054664018,Ben Hall,2024-06-19,D - Non Target,Payment Done,NaN,NaN,0 days 16:31:52,organic,NaN,Web Developer,Morning,2024-06-11 19:39:00,6,1,1000.0,9000.0,5805028000054683001,Zwickau,A2,8.0,50.7194,12.4884,Very slow (>60 min),1000.000000,1000.0
1029,5805028000054257317,Quincy Vincent,2024-06-17,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 00:45:55,facebook ads,NaN,Digital Marketing,Morning,2024-06-10 15:33:00,11,1,1000.0,11000.0,5805028000054284461,Leinfelden-Echterdingen,B1,7.0,48.6941,9.1479,Slow (30-60 min),1000.000000,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21034,5805028000002302112,Julia Nelson,2023-07-19,C - Low,Payment Done,Expensive,performancemax_digitalmarkt_ru_de,0 days 15:24:52,google ads,NaN,Web Developer,Morning,2023-07-18 21:11:00,6,2,1000.0,5000.0,5805028000002302109,Wolfsburg,B1,1.0,52.4226,10.7865,Very slow (>60 min),900.000000,1800.0
21087,5805028000001987082,Julia Nelson,2023-07-17,C - Low,Payment Done,Conditions are not suitable,12.07.2023wide_de,0 days 00:49:16,tiktok ads,NaN,Digital Marketing,Morning,2023-07-17 18:02:00,11,11,1000.0,11000.0,5805028000001986077,NaN,B1,0.0,NaN,NaN,Slow (30-60 min),1000.000000,11000.0
21130,5805028000001885076,Jane Smith,2023-08-31,A - High,Payment Done,NaN,04.07.23recentlymoved_de,0 days 00:13:31,facebook ads,One Payment,Digital Marketing,Morning,2023-07-15 13:27:00,11,11,450.0,4000.0,5805028000001880249,Ingolstadt,NaN,47.0,48.7651,11.4237,Fast (5-15 min),363.636364,4000.0
21265,5805028000001401001,Oliver Taylor,2023-07-16,B - Medium,Payment Done,NaN,02.07.23wide_de,0 days 02:22:36,facebook ads,NaN,Digital Marketing,NaN,2023-07-08 08:56:00,11,8,1000.0,11500.0,5805028000001350049,NaN,NaN,8.0,NaN,NaN,Very slow (>60 min),1043.750000,8350.0


In [56]:
print(deals_df['Revenue_i'].dtype)
print(deals_df['AOV_i'].dtype)

float64
float64


$$
AOV = \frac{\sum Revenue_i}{\sum \text{Months of study}}
$$

In [57]:
aov = (
    deals_df['Revenue_i'].sum(skipna=True) /
    deals_df['Months of study'].sum(skipna=True)
)
print(f"AOV = {aov:.2f}")

AOV = 783.21


In [10]:
contacts_df = pd.read_pickle("contacts_cleaned.pkl")
contacts_df

,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,2023-06-27 11:28:00,2023-12-22 13:34:00
1,5805028000000872003,Charlie Davis,2023-07-03 11:31:00,2024-05-21 10:23:00
2,5805028000000889001,Bob Brown,2023-07-02 22:37:00,2023-12-21 13:17:00
3,5805028000000907006,Bob Brown,2023-07-03 05:44:00,2023-12-29 15:20:00
4,5805028000000939010,Nina Scott,2023-07-04 10:11:00,2024-04-16 16:14:00
...,...,...,...,...
18543,5805028000056889209,Ulysses Adams,2024-06-21 12:11:00,2024-06-21 14:11:00
18544,5805028000056889351,Eva Kent,2024-06-21 13:32:00,2024-06-21 15:32:00
18545,5805028000056892018,Eva Kent,2024-06-21 10:21:00,2024-06-21 12:21:00
18546,5805028000056892055,Yara Edwards,2024-06-21 10:22:00,2024-06-21 12:23:00


In [11]:
spend_df = pd.read_pickle("spend_cleaned.pkl")
spend_df

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup
0,2023-07-03,google ads,gen_analyst_de,6,0.00,0,NaN
1,2023-07-03,google ads,performancemax_eng_de,4,0.01,1,NaN
2,2023-07-03,facebook ads,03.07.23women,187,3.30,6,women
3,2023-07-03,facebook ads,03.07.23women,4,0.02,1,women
4,2023-07-03,facebook ads,02.07.23wide_de,61,0.58,0,wide
...,...,...,...,...,...,...,...
16395,2024-06-21,tiktok ads,22.05.2024wide_de,7,0.03,0,wide
16396,2024-06-21,youtube ads,youtube_shorts_de,90,0.51,2,com_august
16397,2024-06-21,facebook ads,17.03.24wide_at,7,0.07,0,wide
16398,2024-06-21,tiktok ads,12.07.2023wide_de,61,0.16,0,wide


In [12]:
deals_df['Contact Name'].nunique()

18089

In [14]:
deals_df[(deals_df['Months of study'].isna()) & (deals_df['Product'] == 'Find yourself in IT')]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,German level,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i,Revenue_i
3149,5805028000048377930,Quincy Vincent,2024-05-07,D - Non Target,Lost,Duplicate,NaN,NaT,organic,NaN,Find yourself in IT,NaN,2024-05-06 20:29:00,<NA>,<NA>,1.0,NaN,5805028000048410337,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
6042,5805028000042448604,Julia Nelson,2024-05-11,D - Non Target,Payment Done,Expensive,NaN,NaT,organic,One Payment,Find yourself in IT,NaN,2024-04-08 17:34:00,<NA>,<NA>,1.0,1.0,5805028000005448163,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN
9966,5805028000032304003,Charlie Davis,2024-05-09,D - Non Target,Lost,Non target,NaN,NaT,webinar,One Payment,Find yourself in IT,NaN,2024-02-15 19:20:00,<NA>,<NA>,NaN,NaN,5805028000012657064,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN
16120,5805028000017521117,Julia Nelson,2024-05-08,D - Non Target,Lost,Non target,NaN,NaT,webinar,One Payment,Find yourself in IT,NaN,2023-11-17 17:20:00,<NA>,<NA>,NaN,NaN,5805028000005448163,NaN,NaN,173.0,NaN,NaN,NaN,NaN,NaN


## Расчет общей юнит-экономики

In [58]:
# -- поток юнитов масштабирования --
ua = contacts_df['Id'].nunique() 

# -- маректинговый бюджет -- 
ac = spend_df['Spend'].sum()

# -- стоимость привлечения юнита масштабирования --
ltc_cpa = ac / ua

# -- количество клиентов -- 
b = deals_df[
    (deals_df['Stage'] == 'Payment Done') &
    (deals_df['Initial Amount Paid'].notna()) &
    (deals_df['Months of study'].notna())
]['Contact Name'].nunique()

# -- конверсия из лида в клиента -- 
c1 = b / ua

# -- количество транзакций --
t = deals_df[
    (deals_df['Stage'] == 'Payment Done') &
    (deals_df['Initial Amount Paid'].notna()) 
]['Months of study'].sum()

# -- среднее число сделок на одного клиента --
apc = t / b

# -- средняя валовая прибыль на клиента --
cltv = aov * apc

# -- средняя валовая прибыль на юнит масштабирования --
ltv = cltv * c1

# Стоимость привлечения клиента
cac = ac / b

# -- выручка --
revenue = b * aov * apc #deals_df['Revenue_i'].sum(skipna=True)

# -- маржинальная прибыль в юнит-экономике --
cm = ua * (ltv - ltc_cpa)

print(f'''AOV: {aov:.2f}
UA: {ua}
LTC (CPA): {ltc_cpa:.2f}
AC: {ac}
C1: {c1:.4f}
B: {b}
T: {t}
APC: {apc:.2f}
CLTV: {cltv:.2f}
LTV: {ltv:.2f}
CAC: {cac:.2f}
Revenue: {revenue:.2f}
CM: {cm:.2f}
''')

AOV: 783.21
UA: 18548
LTC (CPA): 8.06
AC: 149523.45
C1: 0.0446
B: 827
T: 4561
APC: 5.52
CLTV: 4319.47
LTV: 192.59
CAC: 180.80
Revenue: 3572199.74
CM: 3422676.29



In [59]:
# Соберём всё в датафрейм
metrics = {
    'Metric': [
        'AOV', 'UA', 'LTC (CPA)', 'AC', 'C1', 'B', 'T', 'APC',
        'CLTV', 'LTV', 'CAC', 'Revenue', 'CM'
    ],
    'Value': [
        round(aov, 2), ua, round(ltc_cpa, 2), round(ac, 2), round(c1, 4), b, t,
        round(apc, 2), round(cltv, 2), round(ltv, 2), round(cac, 2),
        round(revenue, 2), round(cm, 2)
    ]
}
metrics_df = pd.DataFrame(metrics)

metrics_df['Value'] = metrics_df['Value'].apply(
    lambda x: f"{x:,.2f}" if isinstance(x, float) else f"{x:,}"
)

In [62]:
metrics_df

,Metric,Value
0,AOV,783.21
1,UA,"18,548.00"
2,LTC (CPA),8.06
3,AC,"149,523.45"
4,C1,0.04
5,B,827.00
6,T,"4,561.00"
7,APC,5.52
8,CLTV,"4,319.47"
9,LTV,192.59


In [63]:
metrics_df.to_pickle('metrics.pkl')

In [64]:
metrics_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
Metric,AOV,UA,LTC (CPA),AC,C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
Value,783.21,"18,548.00",8.06,"149,523.45",0.04,827.00,"4,561.00",5.52,"4,319.47",192.59,180.80,"3,572,199.74","3,422,676.29"


In [65]:
def calculate_unit_economics_by_product(deals_df, contacts_df, spend_df):
        # Общие метрики
    total_ua = contacts_df['Id'].nunique()
    total_ac = spend_df['Spend'].sum()
    total_ltc_cpa = total_ac / total_ua if total_ua != 0 else 0

    results = []
    # Убираем пропуски в Product
    success_deals = deals_df[
        (deals_df['Stage'] == 'Payment Done') &
        (deals_df['Initial Amount Paid'].notna()) &
        (deals_df['Months of study'].notna())
    ].dropna(subset=['Product'])

    grouped = success_deals.groupby('Product', observed=True)

    for product, group in grouped:

        # --- Кол-во клиентов ---
        b = group['Contact Name'].nunique()

        # --- Конверсия ---
        c1 = b / ua if ua != 0 else 0

        # --- Транзакции ---
        t = group['Months of study'].sum()

        # --- APC ---
        apc = t / b if b != 0 else 0

        # --- AOV ---
        total_revenue = group['Revenue_i'].sum()
        aov = total_revenue / t if t != 0 else 0

        # --- CLTV ---
        cltv = aov * apc

        # --- LTV ---
        ltv = cltv * c1

        # --- CAC ---
        cac = ac / b if b != 0 else 0

        # --- Revenue ---
        revenue = b * aov * apc

        # --- CM ---
        cm = ua * (ltv - ltc_cpa)

        results.append({
            'Product': product,
            'AOV': aov,
            'UA': total_ua,
            'LTC (CPA)': total_ltc_cpa,
            'AC': total_ac,
            'C1': c1,
            'B': b,
            'T': t,
            'APC': apc,
            'CLTV': cltv,
            'LTV': ltv,
            'CAC': cac,
            'Revenue': revenue,
            'CM': cm
        })

    df = pd.DataFrame(results)

    # Округляем: всё до 2 знаков, C1 — до 3 знаков
    rounding = {col: 2 for col in df.columns if col != 'Product'}
    rounding['C1'] = 3

    return df.round(rounding)

products_economic = calculate_unit_economics_by_product(deals_df, contacts_df, spend_df)
products_economic

,Product,AOV,UA,LTC (CPA),AC,C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Digital Marketing,783.95,18548,8.06,149523.45,0.025,468,2886,6.17,4834.38,121.98,319.49,2262490.0,2112966.55
1,UX/UI Design,813.37,18548,8.06,149523.45,0.012,226,1170,5.18,4210.82,51.31,661.61,951645.0,802121.55
2,Web Developer,726.10,18548,8.06,149523.45,0.007,135,505,3.74,2716.15,19.77,1107.58,366680.0,217156.55


In [30]:
def simulate_metric_changes_by_product(df, product_name, delta_pct=10):
    increase_factor = 1 + delta_pct / 100
    decrease_factor = 1 - delta_pct / 100

    # Классификация метрик
    metrics = ['AOV', 'UA', 'AC', 'LTC (CPA)', 'C1', 'B', 'T', 'APC', 'CLTV', 'LTV', 'CAC', 'Revenue', 'CM']
    metrics_to_increase = ['AOV', 'UA', 'C1', 'APC']
    metrics_to_decrease = ['LTC (CPA)', 'CAC']
    metrics_fixed = ['AC', 'B', 'T', 'CLTV', 'LTV']  # рассчитываются, но не меняются напрямую
    row_metrics = metrics_to_increase + metrics_to_decrease

    # Получение базовых значений
    base_row = df[df['Product'] == product_name].iloc[0]
    base_values = base_row[metrics].to_dict()

    # Подготовка таблицы
    table = {
        'Metric ↓/ →': ['Base'] + [
            f'{m} +{delta_pct}%' if m in metrics_to_increase else f'{m} -{delta_pct}%' for m in row_metrics
        ]
    }

    for column_metric in metrics:
        values = [base_values[column_metric]]

        for changed_metric in row_metrics:
            temp = base_values.copy()

            # Применение изменений
            if changed_metric in metrics_to_increase:
                temp[changed_metric] *= increase_factor
            elif changed_metric in metrics_to_decrease:
                temp[changed_metric] *= decrease_factor

         # Пересчёт зависимых метрик
            temp['B'] = temp['UA'] * temp['C1']
            temp['T'] = temp['B'] * temp['APC']
            temp['CLTV'] = temp['AOV'] * temp['APC']
            temp['LTV'] = temp['CLTV'] * temp['C1']
            temp['Revenue'] = temp['AOV'] * temp['APC'] * temp['B']
            temp['AC'] = temp['UA'] * temp['LTC (CPA)']
            temp['CM'] = temp['UA'] * (temp['LTV'] - temp['LTC (CPA)'])

            values.append(temp[column_metric])

        table[column_metric] = values
        result_df = pd.DataFrame(table)

    # Округляем: всё до 2 знаков, C1 — до 3 знаков
    rounding = {col: 2 for col in result_df.columns if col != 'Metric ↓/ →'}
    if 'C1' in rounding:
        rounding['C1'] = 3

    return result_df.round(rounding)

    return df.round(rounding)

    return pd.DataFrame(table)

In [31]:
simulate_metric_changes_by_product(products_economic, 'Digital Marketing', delta_pct=10)

,Metric ↓/ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Base,783.95,18548.0,149523.45,8.06,0.025,468.00,2886.00,6.17,4834.38,121.98,319.49,2262490.00,2112966.55
1,AOV +10%,862.35,18548.0,149496.88,8.06,0.025,463.70,2861.03,6.17,5320.67,133.02,319.49,2467194.05,2317697.17
2,UA +10%,783.95,20402.8,164446.57,8.06,0.025,510.07,3147.13,6.17,4836.97,120.92,319.49,2467194.05,2302747.49
3,C1 +10%,783.95,18548.0,149496.88,8.06,0.028,510.07,3147.13,6.17,4836.97,133.02,319.49,2467194.05,2317697.17
4,APC +10%,783.95,18548.0,149496.88,8.06,0.025,463.70,3147.13,6.79,5320.67,133.02,319.49,2467194.05,2317697.17
5,LTC (CPA) -10%,783.95,18548.0,134547.19,7.25,0.025,463.70,2861.03,6.17,4836.97,120.92,319.49,2242903.68,2108356.49
6,CAC -10%,783.95,18548.0,149496.88,8.06,0.025,463.70,2861.03,6.17,4836.97,120.92,287.54,2242903.68,2093406.80


In [34]:
simulate_metric_changes_by_product(products_economic, 'UX/UI Design', delta_pct=10)

,Metric ↓/ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Base,813.37,18548.0,149523.45,8.06,0.012,226.00,1170.00,5.18,4210.82,51.31,661.61,951645.00,802121.55
1,AOV +10%,894.71,18548.0,149496.88,8.06,0.012,222.58,1152.94,5.18,4634.58,55.61,661.61,1031546.78,882049.90
2,UA +10%,813.37,20402.8,164446.57,8.06,0.012,244.83,1268.24,5.18,4213.26,50.56,661.61,1031546.78,867100.21
3,C1 +10%,813.37,18548.0,149496.88,8.06,0.013,244.83,1268.24,5.18,4213.26,55.61,661.61,1031546.78,882049.90
4,APC +10%,813.37,18548.0,149496.88,8.06,0.012,222.58,1268.24,5.70,4634.58,55.61,661.61,1031546.78,882049.90
5,LTC (CPA) -10%,813.37,18548.0,134547.19,7.25,0.012,222.58,1152.94,5.18,4213.26,50.56,661.61,937769.80,803222.61
6,CAC -10%,813.37,18548.0,149496.88,8.06,0.012,222.58,1152.94,5.18,4213.26,50.56,595.45,937769.80,788272.92


In [35]:
simulate_metric_changes_by_product(products_economic, 'Web Developer', delta_pct=10)

,Metric ↓/ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Base,726.10,18548.0,149523.45,8.06,0.007,135.00,505.00,3.74,2716.15,19.77,1107.58,366680.00,217156.55
1,AOV +10%,798.71,18548.0,149496.88,8.06,0.007,129.84,485.59,3.74,2987.18,20.91,1107.58,387842.91,238346.03
2,UA +10%,726.10,20402.8,164446.57,8.06,0.007,142.82,534.15,3.74,2715.61,19.01,1107.58,387842.91,223396.34
3,C1 +10%,726.10,18548.0,149496.88,8.06,0.008,142.82,534.15,3.74,2715.61,20.91,1107.58,387842.91,238346.03
4,APC +10%,726.10,18548.0,149496.88,8.06,0.007,129.84,534.15,4.11,2987.18,20.91,1107.58,387842.91,238346.03
5,LTC (CPA) -10%,726.10,18548.0,134547.19,7.25,0.007,129.84,485.59,3.74,2715.61,19.01,1107.58,352584.46,218037.27
6,CAC -10%,726.10,18548.0,149496.88,8.06,0.007,129.84,485.59,3.74,2715.61,19.01,996.82,352584.46,203087.58


In [ ]:
Расчёт доверительных интервалов для C1 по продуктам

In [66]:
import numpy as np
import scipy.stats as st

# Получаем строку для 'Web Developer'
row = products_economic[products_economic['Product'] == 'Web Developer'].iloc[0]

# Извлекаем значения
p_hat = row['C1']    # наблюдаемая конверсия
n = row['UA']        # Число лидов UA

# Z-значение для 95% доверительного интервала
z = st.norm.ppf(1 - 0.05 / 2) # Z-score для 95% CI

# Стандартная ошибка
se = np.sqrt(p_hat * (1 - p_hat) / n)

# Доверительный интервал
ci_low = p_hat - z * se
ci_high = p_hat + z * se

print(f"95% доверительный интервал для C1: [{ci_low:.5f}, {ci_high:.5f}]")

95% доверительный интервал для C1: [0.00580, 0.00820]


In [67]:
import numpy as np
import scipy.stats as st

# Получаем строку для 'Web Developer'
row = products_economic[products_economic['Product'] == 'Digital Marketing'].iloc[0]

# Извлекаем значения
p_hat = row['C1']    # наблюдаемая конверсия
n = row['UA']        # Число лидов UA

# Z-значение для 95% доверительного интервала
z = st.norm.ppf(1 - 0.05 / 2) # Z-score для 95% CI

# Стандартная ошибка
se = np.sqrt(p_hat * (1 - p_hat) / n)

# Доверительный интервал
ci_low = p_hat - z * se
ci_high = p_hat + z * se

print(f"95% доверительный интервал для C1: [{ci_low:.5f}, {ci_high:.5f}]")

95% доверительный интервал для C1: [0.02275, 0.02725]


In [68]:
import numpy as np
import scipy.stats as st

# Получаем строку для 'Web Developer'
row = products_economic[products_economic['Product'] == 'UX/UI Design'].iloc[0]

# Извлекаем значения
p_hat = row['C1']    # наблюдаемая конверсия
n = row['UA']        # Число лидов UA

# Z-значение для 95% доверительного интервала
z = st.norm.ppf(1 - 0.05 / 2) # Z-score для 95% CI

# Стандартная ошибка
se = np.sqrt(p_hat * (1 - p_hat) / n)

# Доверительный интервал
ci_low = p_hat - z * se
ci_high = p_hat + z * se

print(f"95% доверительный интервал для C1: [{ci_low:.5f}, {ci_high:.5f}]")

95% доверительный интервал для C1: [0.01043, 0.01357]


In [69]:
# 'Web Developer'

from statsmodels.stats.power import NormalIndPower
import numpy as np

p1 = 0.007  # базовая конверсия
p2 = 0.008  # ожидаемая (на 10% больше)

# Пул стандартного отклонения
pooled_std = np.sqrt((p1 * (1 - p1) + p2 * (1 - p2)) / 2)
effect_size = abs(p2 - p1) / pooled_std

# Расчет необходимого размера выборки для каждой группы
analysis = NormalIndPower()
required_n = analysis.solve_power(effect_size=effect_size, power=0.8, alpha=0.05)

print(f"'Web Developer'\nНеобходимое число лидов в каждой группе: {np.ceil(required_n)}")
print(f"Итого лидов для обеих групп: {np.ceil(required_n) * 2}")

'Web Developer'
Необходимое число лидов в каждой группе: 116846.0
Итого лидов для обеих групп: 233692.0


In [71]:
# 'Digital Marketing'


from statsmodels.stats.power import NormalIndPower
import numpy as np

p1 = 0.025  # базовая конверсия
p2 = 0.028  # ожидаемая (на 10% больше)

# Пул стандартного отклонения
pooled_std = np.sqrt((p1 * (1 - p1) + p2 * (1 - p2)) / 2)
effect_size = abs(p2 - p1) / pooled_std

# Расчет необходимого размера выборки для каждой группы
analysis = NormalIndPower()
required_n = analysis.solve_power(effect_size=effect_size, power=0.8, alpha=0.05)

print(f"'Digital Marketing'\nНеобходимое число лидов в каждой группе: {np.ceil(required_n)}")
print(f"Итого лидов для обеих групп: {np.ceil(required_n) * 2}")

'Digital Marketing'
Необходимое число лидов в каждой группе: 44993.0
Итого лидов для обеих групп: 89986.0


In [70]:
# 'UX/UI Design'


from statsmodels.stats.power import NormalIndPower
import numpy as np

p1 = 0.012  # базовая конверсия
p2 = 0.013  # ожидаемая (на 10% больше)

# Пул стандартного отклонения
pooled_std = np.sqrt((p1 * (1 - p1) + p2 * (1 - p2)) / 2)
effect_size = abs(p2 - p1) / pooled_std

# Расчет необходимого размера выборки для каждой группы
analysis = NormalIndPower()
required_n = analysis.solve_power(effect_size=effect_size, power=0.8, alpha=0.05)

print(f"'UX/UI Design'\nНеобходимое число лидов в каждой группе: {np.ceil(required_n)}")
print(f"Итого лидов для обеих групп: {np.ceil(required_n) * 2}")

'UX/UI Design'
Необходимое число лидов в каждой группе: 193765.0
Итого лидов для обеих групп: 387530.0


In [37]:
# def change_metrics_table(df, product, education_type, delta_pct=10):
#     increase_factor = 1 + delta_pct / 100
#     decrease_factor = 1 - delta_pct / 100

#     # Классификация метрик
#     metrics = ['AOV', 'UA', 'AC', 'LTC (CPA)', 'C1', 'B', 'T', 'APC', 'CLTV', 'LTV', 'Revenue', 'CM']
#     metrics_to_increase = ['AOV', 'UA', 'C1', 'APC']
#     metrics_to_decrease = ['LTC (CPA)']
#     metrics_fixed = ['AC', 'B', 'T', 'CLTV', 'LTV']  # рассчитываются, но не меняются напрямую
#     row_metrics = metrics_to_increase + metrics_to_decrease

#     # Получение базовых значений
#     base = df[
#         (df['Product'] == product) &
#         (df['Education Type'] == education_type)
#     ].iloc[0]
#     base_values = base[metrics].to_dict()

#     # Подготовка таблицы
#     table = {
#         'Метрика ↑ / ↓ Метрика →': ['Base'] + [
#             f'{m} +{delta_pct}%' if m in metrics_to_increase else f'{m} -{delta_pct}%' for m in row_metrics
#         ]
#     }

#     for col_metric in metrics:
#         col_values = [base_values[col_metric]]

#         for row_metric in row_metrics:
#             temp = base_values.copy()

#             # Изменение выбранной метрики
#             if row_metric in metrics_to_increase:
#                 temp[row_metric] *= increase_factor
#             elif row_metric in metrics_to_decrease:
#                 temp[row_metric] *= decrease_factor

#             # Пересчёт зависимых метрик
#             temp['CLTV'] = temp['AOV'] * temp['APC']
#             temp['LTV'] = temp['CLTV'] * temp['C1']
#             temp['CM'] = temp['UA'] * (temp['LTV'] - temp['LTC (CPA)'])
#             temp['Revenue'] = temp['AOV'] * temp['APC'] * temp['B']

#             col_values.append(temp[col_metric])

#         table[col_metric] = col_values

#     return pd.DataFrame(table)

In [38]:
change_metrics_table(unit_df, 'Digital Marketing', 'Evening', delta_pct=10)
# pd.set_option('display.float_format', '{:,.2f}'.format)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","129,866.55"
1,AOV +10%,402.26,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,744.01",16.57,"307,329.00","157,805.55"
2,UA +10%,365.69,"20,402.80","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","142,853.20"
3,C1 +10%,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",16.57,"279,390.00","157,805.55"
4,APC +10%,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,7.50,"2,744.01",16.57,"307,329.00","157,805.55"
5,LTC (CPA) -10%,365.69,"18,548.00","149,523.45",7.26,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","144,818.89"


In [39]:
change_metrics_table(unit_df, 'Digital Marketing', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,765,126.55"
1,AOV +10%,"1,024.38","18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"6,034.71",113.55,"2,106,115.00","1,956,591.55"
2,UA +10%,931.25,"20,402.80","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,941,639.21"
3,C1 +10%,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",113.55,"1,914,650.00","1,956,591.55"
4,APC +10%,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,6.48,"6,034.71",113.55,"2,106,115.00","1,956,591.55"
5,LTC (CPA) -10%,931.25,"18,548.00","149,523.45",7.26,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,780,078.89"


In [40]:
change_metrics_table(unit_df, 'UX/UI Design', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","679,186.55"
1,AOV +10%,"1,028.87","18,548.00","149,523.45",8.06,0.01,169,886,5.24,"5,393.97",49.15,"911,581.00","762,057.55"
2,UA +10%,935.34,"20,402.80","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","747,105.21"
3,C1 +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",49.15,"828,710.00","762,057.55"
4,APC +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.77,"5,393.97",49.15,"911,581.00","762,057.55"
5,LTC (CPA) -10%,935.34,"18,548.00","149,523.45",7.26,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","694,138.90"


In [41]:
change_metrics_table(unit_df, 'UX/UI Design', 'Evening', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,432.87,"18,548.00","149,523.45",8.06,0.00,58,284,4.90,"2,119.57",6.63,"122,935.00","-26,588.45"
1,AOV +10%,476.16,"18,548.00","149,523.45",8.06,0.00,58,284,4.90,"2,331.53",7.29,"135,228.50","-14,294.95"
2,UA +10%,432.87,"20,402.80","149,523.45",8.06,0.00,58,284,4.90,"2,119.57",6.63,"122,935.00","-29,247.30"
3,C1 +10%,432.87,"18,548.00","149,523.45",8.06,0.00,58,284,4.90,"2,119.57",7.29,"122,935.00","-14,294.95"
4,APC +10%,432.87,"18,548.00","149,523.45",8.06,0.00,58,284,5.39,"2,331.53",7.29,"135,228.50","-14,294.95"
5,LTC (CPA) -10%,432.87,"18,548.00","149,523.45",7.26,0.00,58,284,4.90,"2,119.57",6.63,"122,935.00","-11,636.11"


In [42]:
change_metrics_table(unit_df, 'UX/UI Design', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","679,186.55"
1,AOV +10%,"1,028.87","18,548.00","149,523.45",8.06,0.01,169,886,5.24,"5,393.97",49.15,"911,581.00","762,057.55"
2,UA +10%,935.34,"20,402.80","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","747,105.21"
3,C1 +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",49.15,"828,710.00","762,057.55"
4,APC +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.77,"5,393.97",49.15,"911,581.00","762,057.55"
5,LTC (CPA) -10%,935.34,"18,548.00","149,523.45",7.26,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","694,138.90"


In [43]:
change_metrics_table(unit_df, 'Web Developer', 'Morning', delta_pct=10)	

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","217,156.55"
1,AOV +10%,798.71,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,987.76",21.75,"403,348.00","253,824.55"
2,UA +10%,726.10,"20,402.80","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","238,872.21"
3,C1 +10%,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,3.74,"2,716.15",21.75,"366,680.00","253,824.55"
4,APC +10%,726.10,"18,548.00","149,523.45",8.06,0.01,135,505,4.11,"2,987.76",21.75,"403,348.00","253,824.55"
5,LTC (CPA) -10%,726.10,"18,548.00","149,523.45",7.26,0.01,135,505,3.74,"2,716.15",19.77,"366,680.00","232,108.90"
